In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)


Using matplotlib backend: Qt5Agg


In [4]:
#Organizar los datos en directorios de prueba, válidacion y de entrenamiento
#os.chdir('./archive')
#devuelve el directorio de trabajo actual.
os.getcwd()

'C:\\Users\\LENOVO\\OneDrive\\Python\\Proyectos\\Face Recognition with Tensorflow and Keras\\archive'

In [50]:
#dento del archive no hay directoriso creado, asi que lo creamos y copiamos una cierta cantidad de imagenes
if os.path.isdir('train/cesar') is False:
    os.makedirs('train/cesar')
    os.makedirs('train/brayan')
    os.makedirs('train/ronald')

    os.makedirs('valid/cesar')
    os.makedirs('valid/brayan')
    os.makedirs('valid/ronald')
   
    os.makedirs('test/cesar')
    os.makedirs('test/brayan')
    os.makedirs('test/ronald')
    
    for c in random.sample(glob.glob('cesar*'), 225):
        shutil.move(c, 'train/cesar')
    for c in random.sample(glob.glob('brayan*'), 225):
        shutil.move(c, 'train/brayan')
    for c in random.sample(glob.glob('ronald*'), 225):
        shutil.move(c, 'train/ronald')
    
        
    for c in random.sample(glob.glob('cesar*'), 45):
        shutil.move(c, 'valid/cesar')
    for c in random.sample(glob.glob('brayan*'), 45):
        shutil.move(c, 'valid/brayan')
    for c in random.sample(glob.glob('ronald*'), 45):
        shutil.move(c, 'valid/ronald')
   
        
        
    for c in random.sample(glob.glob('cesar*'), 30):
        shutil.move(c, 'test/cesar')
    for c in random.sample(glob.glob('brayan*'), 30):
        shutil.move(c, 'test/brayan')
    for c in random.sample(glob.glob('ronald*'), 30):
        shutil.move(c, 'test/ronald')
  
    


In [5]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

In [6]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cesar', 'brayan', 'ronald'], batch_size=10);
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cesar', 'brayan', 'ronald'], batch_size=10);
#El shuffle false conservara el orden de su conjunto de datos tras cada epoch, en cambio por defectvo esto es verdadero
# xq Keras barajará los datos de entrenamiento antes de cada época 
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cesar', 'brayan', 'ronald'], batch_size=10, shuffle=False);

Found 675 images belonging to 3 classes.
Found 135 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [53]:
len(train_batches)

68

In [65]:

plt.imshow(train_batches[66][0][5])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [66]:
imgs, labels = next(train_batches)

In [67]:
#esta función trazará imágenes en forma de cuadrícula con 1 fila y 10 columnas donde se colocan las imágenes
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [68]:
plotImages(imgs)
print(labels)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


# Entrenar Modelo

In [69]:
# Descargar el modelo de internet
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [70]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [71]:
#Convertir el modelo vgg16 a un modelo secuencial
model = Sequential()
#recorrer cad capa del modelo vgg16 excepto la ultima que es la capa de salida
for layer in vgg16_model.layers[:-1]:
    model.add(layer)


In [72]:
    
for layer in model.layers:
    layer.trainable = False

In [73]:
model.add(Dense(units=3, activation="softmax"))


In [74]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [75]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [76]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

Epoch 1/5
68/68 - 325s - loss: 0.5954 - accuracy: 0.7689 - val_loss: 0.1858 - val_accuracy: 0.9926
Epoch 2/5
68/68 - 301s - loss: 0.1281 - accuracy: 0.9911 - val_loss: 0.0741 - val_accuracy: 1.0000
Epoch 3/5
68/68 - 309s - loss: 0.0633 - accuracy: 0.9970 - val_loss: 0.0420 - val_accuracy: 1.0000
Epoch 4/5
68/68 - 293s - loss: 0.0401 - accuracy: 1.0000 - val_loss: 0.0274 - val_accuracy: 1.0000
Epoch 5/5
68/68 - 252s - loss: 0.0286 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 1.0000


# Predictions

In [15]:
from tensorflow.keras.models import load_model


In [16]:
model = load_model('../Models/model_vgg16.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [17]:
predictions = model.predict(x=test_batches, verbose=0)

In [19]:
test_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [20]:
np.argmax(predictions, axis=-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [21]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [22]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta función imprime y traza la matriz de confusión.
     La normalización se puede aplicar configurando `normalize = True`. .
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print("Matriz de confusión, sin normalización")
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Observación')
    plt.xlabel('Predicción')

In [23]:
cm_plot_labels=['cesar', 'brayan', 'ronald']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title="Confusion Matrix")

Matriz de confusión, sin normalización
[[30  0  0]
 [ 0 30  0]
 [ 0  0 30]]


# Guardar Modelo

In [84]:
os.getcwd()

'C:\\Users\\LENOVO\\OneDrive\\Python\\Proyectos\\Face Recognition with Tensorflow and Keras\\archive'

In [85]:
if os.path.isfile('../Models/model_vgg16.h5') is False:
    model.save('../Models/model_vgg16.h5')
else:
    model.save()